# 🧩 1. Setup inicial


In [18]:
# 1.1 Cargar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from autogluon.tabular import TabularPredictor
import os
import glob
from datetime import datetime

# 1.2 Paths de entrada y salida
ruta_parquet = "C:/Developer/Laboratorio_III/data/dataset_product_periodo_con_clusters.parquet"
output_dir = "output_forecasts_by_cluster_tabular"
os.makedirs(output_dir, exist_ok=True)

# 1.3 Archivo de log
log_file = open("log_forecast_clusters.txt", "w")
def log(msg):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{timestamp}] {msg}\\n"
    log_file.write(line)
    log_file.flush()
    print(line)


# 📥 2. Cargar dataset y clusters

In [19]:
# 2.1 Cargar dataset con clusters
df = pd.read_parquet(ruta_parquet)
# Normalizar fecha
df['fecha'] = pd.to_datetime(df['fecha']).dt.normalize()

# 2.2 Crear campo clase (tn_total desplazado +2)
df['clase'] = df.groupby('product_id')['tn_total'].shift(-2)

# 2.3 Revisar clusters únicos
clusters_unicos = df['cluster_dtw'].dropna().unique()
log(f"Clusters únicos: {clusters_unicos}")

[2025-07-06 21:13:41] Clusters únicos: [ 1  0 31  4 39  7 43 48  9 41 24 45 35 10 20 18 32 12 15 21 44 19 30 47
 46 37 17 25  3 26 38 22  5  2 42 14 34  8 27 13 23 36 49 29 33 28 16 40
 11  6]\n


# 🔄 3. Loop por cluster

In [ ]:
# 3.1 Iterar cluster por cluster
for cluster_id in clusters_unicos:
    log(f"===> Iniciando procesamiento Cluster {cluster_id}")
    cluster_start = datetime.now()
    
    # 1. Filtrar cluster
    df_cluster = df[df['cluster_dtw'] == cluster_id].copy()
    if df_cluster.empty:
        log(f"Cluster {cluster_id} está vacío. Saltando...")
        continue
    
    # 2. Ingeniería de features (misma lógica)
    for lag in range(1, 37):
        df_cluster[f'tn_{lag}'] = df_cluster.groupby('product_id')['tn_total'].shift(lag)
        df_cluster[f'diff_tn_{lag}'] = df_cluster['clase'] - df_cluster[f'tn_{lag}']
    
    df_cluster['rollmean_3'] = df_cluster.groupby('product_id')['tn_total'].transform(lambda x: x.shift(1).rolling(3).mean())
    df_cluster['rollmean_6'] = df_cluster.groupby('product_id')['tn_total'].transform(lambda x: x.shift(1).rolling(6).mean())
    df_cluster['rollmean_9'] = df_cluster.groupby('product_id')['tn_total'].transform(lambda x: x.shift(1).rolling(9).mean())
    df_cluster['rollmean_12'] = df_cluster.groupby('product_id')['tn_total'].transform(lambda x: x.shift(1).rolling(12).mean())
    df_cluster['diff_rollmean_12'] = df_cluster['clase'] - df_cluster['rollmean_12']

    # Factorizar no numéricas
    for col in df_cluster.select_dtypes(include='object').columns:
        if col != 'cluster_dtw':
            df_cluster[col + '_factorized'], _ = pd.factorize(df_cluster[col])

    log(f"Features OK para Cluster {cluster_id}")
    
    # 3. Train/Test split (sin validación ni escalado)
    df_cluster['fecha'] = pd.to_datetime(df_cluster['fecha']).dt.normalize()
    
    train_set = df_cluster[(df_cluster['fecha'] <= '2019-10-01') & df_cluster['clase'].notnull()].copy()
    test_set  = df_cluster[df_cluster['fecha'] == '2019-12-01'].copy()
    
    log(f"Train shape: {train_set.shape} | Test shape: {test_set.shape}")
    
    if train_set.empty or test_set.empty:
        log(f"Cluster {cluster_id} sin datos suficientes. Saltando...")
        continue
    
    # 4. Train modelo tabular (sin escalado)
    features = ['tn_total'] + [f'tn_{i}' for i in range(1, 37)] + \
               [f'diff_tn_{i}' for i in range(1, 37)] + \
               ['rollmean_3', 'rollmean_6', 'rollmean_9', 'rollmean_12', 'diff_rollmean_12'] + \
               [c for c in df_cluster.columns if c.endswith('_factorized')] + ['product_id']

    predictor = TabularPredictor(label='clase', problem_type='regression', eval_metric='mae')
    predictor.fit(train_data=train_set[features + ['clase']], time_limit=1200, presets='high_quality')

    # 5. Predicción final sin inversa
    test_set['tn'] = predictor.predict(test_set[features])
    test_set['tn'] = test_set['tn'].clip(lower=0)

    # 6. Guardar CSV por cluster
    test_set[['product_id', 'tn']].to_csv(f"{output_dir}/forecast_cluster_{cluster_id}.csv", index=False)
    log(f"✅ Cluster {cluster_id}: Predicciones guardadas | Productos: {test_set['product_id'].nunique()}")
    
    cluster_end = datetime.now()
    duration = (cluster_end - cluster_start).total_seconds() / 60
    log(f"===> Cluster {cluster_id} completado en {duration:.2f} min.")




No path specified. Models will be saved in: "AutogluonModels\ag-20250707_001345"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.21
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          14
Memory Avail:       2.65 GB / 15.31 GB (17.3%)
Disk Space Avail:   216.76 GB / 475.95 GB (45.5%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "c:\Developer\Laboratorio_III\notebooks\AutogluonModels\ag-20250707_001345"
Train Data Rows:    563
Train Data Columns: 87
Label Column:       clase
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2711.89 MB
	Train Data (Original)  Memory Usage: 0.37 MB (0.0% of available memory)
	Inferring data type of each feature ba

[2025-07-06 21:13:45] ===> Iniciando procesamiento Cluster 1\n
[2025-07-06 21:13:45] Features OK para Cluster 1\n
[2025-07-06 21:13:45] Train shape: (563, 100) | Test shape: (17, 100)\n


	Useless Original Features (Count: 6): ['tn_34', 'tn_35', 'tn_36', 'diff_tn_34', 'diff_tn_35', 'diff_tn_36']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 1): ['mm-yyyy_factorized']
		These features were not used to generate any of the output features. Add a feature generator compatible with these features to utilize them.
		Features can also be unused if they carry very little information, such as being categorical but having almost entirely unique values or being duplicates of other features.
		These features do not need to be present at inference time.
		('int', []) : 1 | ['mm-yyyy_factorized']
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 72 | ['tn_total', 'tn_1', 'tn_2', 'tn_3', 'tn_4', ...]
		('int', [])   :  8 | ['periodo_factorized', 'cat1_facto

[1000]	valid_set's l1: 16.2201


	-16.2058	 = Validation score   (-mean_absolute_error)
	3.08s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 295.09s of the 295.09s of remaining time.
	-19.144	 = Validation score   (-mean_absolute_error)
	0.62s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 294.41s of the 294.41s of remaining time.
	-18.9134	 = Validation score   (-mean_absolute_error)
	30.36s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 264.03s of the 264.03s of remaining time.
	-20.978	 = Validation score   (-mean_absolute_error)
	0.49s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 263.49s of the 263.49s of remaining time.
		Import fastai failed. A quick tip is to install via `pip install autogluon.tabular[fastai]==1.3.1`. 
Fitting model: XGBoost ... Training model for up

...

[2025-07-06 22:07:14] ✅ Cluster 40: Predicciones guardadas | Productos: 4\n
[2025-07-06 22:07:14] ===> Cluster 40 completado en 1.53 min.\n
[2025-07-06 22:07:14] ===> Iniciando procesamiento Cluster 11\n
[2025-07-06 22:07:14] Features OK para Cluster 11\n
[2025-07-06 22:07:14] Train shape: (203, 100) | Test shape: (6, 100)\n


	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 72 | ['tn_total', 'tn_1', 'tn_2', 'tn_3', 'tn_4', ...]
		('int', [])   :  5 | ['periodo_factorized', 'cat2_factorized', 'descripcion_factorized', 'quarter_factorized', 'product_id']
	0.1s = Fit runtime
	77 features in original data used to generate 77 features in processed data.
	Train Data (Processed) Memory Usage: 0.12 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.14s ...
AutoGluon will gauge predictive performance using evaluation metric: 'mean_absolute_error'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.2, Train Rows: 162, Val Rows: 41
User-specified model hyperparameters to be fit:
{
	'NN_TORCH': [{}],
	'GBM': [{'extra_trees': True, 'a

[2025-07-06 22:07:23] ✅ Cluster 11: Predicciones guardadas | Productos: 6\n
[2025-07-06 22:07:23] ===> Cluster 11 completado en 0.14 min.\n
[2025-07-06 22:07:23] ===> Iniciando procesamiento Cluster 6\n
[2025-07-06 22:07:23] Features OK para Cluster 6\n
[2025-07-06 22:07:23] Train shape: (106, 100) | Test shape: (9, 100)\n


Fitting model: KNeighborsUnif ... Training model for up to 299.90s of the 299.90s of remaining time.
	-0.3686	 = Validation score   (-mean_absolute_error)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: KNeighborsDist ... Training model for up to 299.89s of the 299.88s of remaining time.
	-0.3223	 = Validation score   (-mean_absolute_error)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMXT ... Training model for up to 299.87s of the 299.87s of remaining time.
	-0.1931	 = Validation score   (-mean_absolute_error)
	0.63s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 299.23s of the 299.22s of remaining time.
	-0.2223	 = Validation score   (-mean_absolute_error)
	0.61s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 298.60s of the 298.60s of remaining time.
	-0.3375	 = Validation score   (-mean_absolute_error)
	0.45s

[2025-07-06 22:07:28] ✅ Cluster 6: Predicciones guardadas | Productos: 9\n
[2025-07-06 22:07:28] ===> Cluster 6 completado en 0.09 min.\n


# 🗃️ 4. Combinar todos los resultados

In [21]:
# 4.1 Combinar todos los CSV
csv_files = glob.glob(f"{output_dir}/forecast_cluster_*.csv")
dfs = [pd.read_csv(f) for f in csv_files]
df_final = pd.concat(dfs, axis=0)

df_final.to_csv("forecast_tabular_clusters_202002.csv", index=False)
log(f"Archivo final forecast_tabular_clusters_202002.csv generado.")

log_file.close()


[2025-07-06 22:10:00] Archivo final forecast_tabular_clusters_202002.csv generado.\n
